# Signal Reconstruction

## Theoretical Approach
In this final stage of the pipeline, we transform the processed frequency-domain data back into the time domain. This process is known as **Signal Reconstruction**.

The goal is to take the filtered spectral coefficients and compute the corresponding time-domain signal. Because the Discrete Fourier Transform represents a bijective mapping between the time and frequency bases, this operation is perfectly invertible. However, since we modified the coefficients via thresholding, the reconstructed signal $\mathbf{\hat{x}}$ will be a denoised approximation of the original input $\mathbf{x}$.

We will implement the **Inverse Discrete Fourier Transform** from scratch using Linear Algebra. By constructing the inverse basis matrix, we demonstrate the orthogonality properties of the Fourier basis vectors.

## Mathematical Formulation

We aim to recover the time-domain signal vector $\mathbf{x}$ from the frequency-domain vector $\mathbf{X}$. The inverse operation is defined as:

$$
\mathbf{x} = F^{-1} \cdot \mathbf{X}
$$

### The Inverse Fourier Matrix
The forward Fourier matrix $F$ is unitary, meaning its columns are orthogonal. A fundamental property of the Fourier matrix is that its inverse is proportional to its **Hermitian Transpose**, denoted as $F^H$.

The orthogonality relation is given by:
$$
F^H F = N I
$$
Where $I$ is the identity matrix and $N$ is the number of samples. Therefore, the inverse matrix is:

$$
F^{-1} = \frac{1}{N} F^H
$$

### Element-wise Definition
While the forward matrix $F$ uses negative exponents ($\omega_N = e^{-i \frac{2\pi}{N}}$), the conjugate transpose $F^H$ utilizes the complex conjugates, effectively flipping the sign of the exponent to positive.

The element in the $j$-th row and $k$-th column of the Inverse Matrix before scaling is:

$$
(F^H)_{j,k} = \omega_N^{-jk} = e^{i \frac{2\pi}{N} j k}
$$

This results in the following structure for the reconstruction:

$$
\mathbf{x}[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] \cdot e^{i \frac{2\pi}{N} k n}
$$

### Libraries Import

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### Data Processing

In [ ]:
# Loading dataframes
df_freq = pd.read_csv("frequency_data.csv")
df_orig = pd.read_csv("signal_data.csv")

# Reconstructing the complex vector X
X_vector = df_freq["X_real"].values + 1j * df_freq["X_imag"].values

# Extracting parameters
N = len(X_vector)
time_t = df_orig['time_s'].values

print(f"Loaded spectral data with N={N} coefficients.")
print("Sample of complex vector X:", X_vector[:3])

### Inverse Fourier Matrix Construction

In [ ]:
def create_idft_matrix(n_samples):
    
    # Defining grid
    j = np.arange(n_samples).reshape(-1, 1)
    k = np.arange(n_samples).reshape(1, -1)

    # Calculating the exponent constant
    exponent_constant = 2j * np.pi / n_samples

    # Creating the matrix
    F_conjugate = np.exp(exponent_constant * j * k)

    # 4. Apply Normalization Factor 1/N
    F_inv = F_conjugate / n_samples

    return F_inv

# Generate the Inverse Matrix
F_inv_matrix = create_idft_matrix(N)

print(f"IDFT Matrix Shape: {F_inv_matrix.shape}")
print("First 2x2 corner of Matrix F_inv:")
print(np.round(F_inv_matrix[:2, :2], 3))